In [1]:
import requests, json
import pandas as pd
from useful_funcs import *
from predict_player_scores import get_latest_players_df

In [2]:
league_url = 'https://draft.premierleague.com/api/league/19188/element-status'
r = requests.get(league_url).json()
ownership_df = pd.json_normalize(r['element_status'])

In [3]:
url = 'https://draft.premierleague.com/api/bootstrap-static'
req = requests.get(url).json()
players_df = pd.json_normalize(req['elements'])

In [4]:
players_df['full_name'] = players_df.apply(lambda row: combine_clean_names(row['first_name'], row['second_name']), axis=1)
merge_name_df = players_df[['id', 'full_name']]

In [5]:
ownership_df_name = ownership_df.merge(merge_name_df, how='left', left_on='element', right_on='id') 

In [6]:
old_pred_df = pd.read_csv('../drafting/player_score_predictions_delta.csv')

In [7]:
old_pred_df['full_name'] = old_pred_df['full_name'].str.lower().str.strip()

In [8]:
pred_df = old_pred_df.merge(ownership_df_name, how='outer', on='full_name')

In [9]:
pred_df

,full_name,pos,team_name,total_points,y_pred,notes,y_pred_delta_1,y_pred_delta_2,y_pred_delta_3,y_pred_delta_4,...,y_pred_delta_7,y_pred_delta_8,y_pred_delta_9,y_pred_delta_10,is_available,element,in_accepted_trade,owner,status,id
0,aaron_anselmino,DEF,Chelsea,0.0,-1.800599,NaN,0.625922,0.0,0.000000,0.000000,...,0.000000,8.871476,0.000000,0.000000,True,234.0,False,NaN,a,234.0
1,aaron_hickey,DEF,Brentford,0.0,5.340009,NaN,0.000000,0.0,0.000000,0.611548,...,0.000000,0.000000,0.000000,0.000000,True,116.0,False,NaN,a,116.0
2,aaron_ramsdale,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,673.0,False,NaN,a,673.0
3,aaron_ramsey,MID,Burnley,0.0,27.162412,promoted_team,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,True,211.0,False,NaN,a,211.0
4,aaron_wan_bissaka,DEF,West Ham,118.0,56.569787,NaN,0.000000,0.0,0.000000,0.000000,...,1.872975,0.000000,0.000000,0.000000,True,610.0,False,NaN,a,610.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
736,zain_silcott_duberry,MID,Bournemouth,1.0,24.922663,jan_transfer,0.086670,0.0,0.000000,0.000000,...,0.000000,0.000000,0.421542,0.000000,True,95.0,False,NaN,a,95.0
737,zak_johnson,DEF,Sunderland,0.0,33.661237,promoted_team,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.625922,True,538.0,False,NaN,a,538.0
738,zeki_amdouni,FWD,Burnley,0.0,66.568110,promoted_team,0.000000,0.0,0.000000,0.970461,...,5.550651,1.220135,0.000000,0.356870,True,216.0,False,NaN,a,216.0
739,zepiqueno_redmond,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,65.0,False,NaN,a,65.0


In [10]:
pos_pred_df2 = pred_df.sort_values('y_pred', ascending=False).query('status=="a"')

pos_pred_df3 = pos_pred_df2.iloc[:, :5]

In [11]:
updated_player_df = get_latest_players_df()

col_of_interest = ['first_name', 'second_name',
                   'form', 'total_points', 'transfers_in', 'transfers_out', 'value_form', 'minutes']

current_stats = updated_player_df[col_of_interest]

current_stats['full_name'] = updated_player_df.apply(lambda row: combine_clean_names(row['first_name'], row['second_name']), axis=1)

current_stats2 = current_stats.drop(['first_name', 'second_name'], axis=1)

current_stats2['transfer_delta'] = current_stats2['transfers_in'] - current_stats2['transfers_out']

C:\Users\dellis2\AppData\Local\Temp\ipykernel_39796\1447550088.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_stats['full_name'] = updated_player_df.apply(lambda row: combine_clean_names(row['first_name'], row['second_name']), axis=1)


In [12]:
pos_pred_df4 = pos_pred_df3.merge(current_stats2, on='full_name', how='left')

In [13]:
pos_pred_df4.query('pos=="MID"').head(30)

,full_name,pos,team_name,total_points_x,y_pred,form,total_points_y,transfers_in,transfers_out,value_form,minutes,transfer_delta
0,omar_marmoush,MID,Man City,73.0,166.071973,2.7,9,161912,985415,0.3,148,-823503
1,son_heung_min,MID,Spurs,129.0,165.209198,0.0,0,0,23340,0.0,0,-23340
2,matheus_santos_carneiro_da_cunha,MID,Man Utd,178.0,153.754197,1.0,5,720023,1152302,0.1,210,-432279
3,luis_diaz_marulanda,MID,Liverpool,183.0,144.874270,0.0,0,0,6596,0.0,0,-6596
6,marcus_rashford,MID,Man Utd,95.0,117.646442,0.0,0,0,1705,0.0,0,-1705
7,justin_kluivert,MID,Bournemouth,158.0,115.192051,1.0,3,34466,63491,0.1,44,-29025
8,leandro_trossard,MID,Arsenal,152.0,114.051526,1.7,5,4257,19041,0.2,48,-14784
9,gabriel_martinelli_silva,MID,Arsenal,125.0,114.051526,1.3,5,21890,77347,0.2,140,-55457
10,noni_madueke,MID,Arsenal,120.0,114.051526,2.7,9,125485,70209,0.4,259,55276
12,savio_moreira_de_oliveira,MID,Man City,106.0,111.910340,0.3,1,2953,73764,0.0,13,-70811


In [15]:
pos_pred_df4.sort_values('transfers_in', ascending=False).query('pos=="MID"').head(50)

,full_name,pos,team_name,total_points_x,y_pred,form,total_points_y,transfers_in,transfers_out,value_form,minutes,transfer_delta
152,tijjani_reijnders,MID,Man City,0.0,57.748707,2.3,17,2018822,892231,0.4,360,1126591
2,matheus_santos_carneiro_da_cunha,MID,Man Utd,178.0,153.754197,1.0,5,720023,1152302,0.1,210,-432279
440,josh_king,MID,Fulham,10.0,24.835993,1.7,7,407116,126121,0.4,271,280995
70,lucas_tolentino_coelho_de_lima,MID,West Ham,80.0,79.927380,7.0,23,390610,141446,1.2,337,249164
296,kiernan_dewsbury_hall,MID,Chelsea,18.0,41.865565,5.3,18,293215,231697,1.1,360,61518
17,iliman_ndiaye,MID,Everton,114.0,101.479645,6.7,22,271794,539495,1.0,333,-267701
255,josh_cullen,MID,Burnley,0.0,45.216290,6.3,23,259035,83404,1.3,332,175631
139,martin_zubimendi_ibanez,MID,Arsenal,0.0,59.889894,6.7,25,242051,137902,1.2,360,104149
126,marcus_tavernier,MID,Bournemouth,87.0,61.030419,5.3,20,206192,81629,1.0,332,124563
343,pape_matar_sarr,MID,Spurs,71.0,38.832055,4.3,19,199358,160552,0.9,345,38806
